In [1]:
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
from sklearn.metrics import pairwise_distances
import numpy as onp
import jax
import jax.numpy as np
from jax import jit, vmap
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm as t
from ipywidgets import interact
from jax import grad
from jax.scipy.optimize import minimize
from jax import config
config.update("jax_enable_x64", True)

from jax.scipy.linalg import cholesky, cho_factor, cho_solve
from jax.scipy.optimize import minimize
from jaxopt import ProjectedGradient
from jaxopt.projection import projection_box

import pandas as pd

In [2]:
import pandas as pd

In [3]:
PCANet = pd.read_csv("data/PCA_NS.csv", header = None).to_numpy()
DeepONet = pd.read_csv("data/DeepONet_NS.csv", header = None).to_numpy()
Para = pd.read_csv("data/PARA_NS.csv", header = None).to_numpy()
FNO = pd.read_csv("data/FNO_NS.csv", header = None).to_numpy()

FileNotFoundError: [Errno 2] No such file or directory: 'data/PCA_NS.csv'

In [ ]:
np.min(PCANet[:,4]), np.min(FNO[:,4]), np.min(DeepONet[:, 4]), np.min(Para[:, 4])

(DeviceArray(0.02654266, dtype=float64),
 DeviceArray(0.00259586, dtype=float64),
 DeviceArray(0.03629472, dtype=float64),
 DeviceArray(0.04088747, dtype=float64))

In [5]:
#The columns = [Nexamples, network width, Train, Test]

In [4]:
Inputs = np.load('../navier_stokes_data/NavierStokes_inputs.npy')
Outputs = np.load('../navier_stokes_data/NavierStokes_outputs.npy')

In [5]:
Inputs.shape

(64, 64, 40000)

In [6]:
Inputs = Inputs.transpose((2,1,0))
Outputs = Outputs.transpose((2,1,0))

Inputs_fl = Inputs.reshape(len(Inputs), 64*64)
Outputs_fl = Outputs.reshape(len(Outputs), 64*64)

Linear regression

In [9]:
results = []
for Ntrain in [20000]:
    for N_components in [128, 256, 512, 1024]:
        print(Ntrain, N_components)
        Ytr = Outputs_fl[:Ntrain]
        Xtr = Inputs_fl[:Ntrain]
        pca = PCA(n_components=min(N_components,Ntrain))
        Xtr = pca.fit_transform(Xtr)
        reg = LinearRegression(n_jobs = -1).fit(Xtr, Ytr)
        #Ypred Ypredtr = reg.predict(Xtr)
        Ypredtr = reg.predict(Xtr)
        train_error = np.mean(np.linalg.norm(Ypredtr-Ytr, axis = 1)/np.linalg.norm(Ytr, axis = 1))
        Xtest = Inputs_fl[20000:]
        Ytest = Outputs_fl[20000:]
        Xtest = pca.transform(Xtest)
        Ypred = reg.predict(Xtest)
        test_error = np.mean(np.linalg.norm(Ypred-Ytest, axis = 1)/np.linalg.norm(Ytest, axis = 1))
        print(train_error, test_error)
        results.append([Ntrain, N_components, train_error, test_error])

results = np.array(results)

20000 128
0.053725325383721885 0.054129107629736904
20000 256
0.053724771853986414 0.0541287837480367
20000 512
0.053725442394692256 0.054128941233027035
20000 1024
0.05372536128518232 0.0541288361220119


GP regression

In [10]:
# from sklearn.gaussian_process import GaussianProcessRegressor
# from sklearn.gaussian_process.kernels import Matern, RBF, RationalQuadratic

# kernel = Matern(nu = 2.5)

# Xtr.shape

# resultsgp = []
# for Ntrain in [156, 312, 624, 1250, 2500]:
#     print(Ntrain)
#     Ytr = Outputs_fl[:Ntrain]
#     Xtr = Inputs_fl[:Ntrain]
#     pca = PCA(n_components=128)
#     Xtr = pca.fit_transform(Xtr)
    
#     model = GaussianProcessRegressor(kernel, alpha = 1e-10)
#     model.fit(Xtr, Ytr)
#     #Ypred Ypredtr = reg.predict(Xtr)
#     Ypredtr = model.predict(Xtr)
#     train_error = np.mean(np.linalg.norm(Ypredtr-Ytr, axis = 1)/np.linalg.norm(Ytr, axis = 1))
#     Xtest = Inputs_fl[20000:]
#     Ytest = Outputs_fl[20000:]
#     Xtest = pca.transform(Xtest)
#     Ypred= model.predict(Xtest)
#     test_error = np.mean(np.linalg.norm(Ypred-Ytest, axis = 1)/np.linalg.norm(Ytest, axis = 1))
#     print(train_error, test_error)
#     resultsgp.append([Ntrain, train_error, test_error])

In [11]:
#Jax regression

In [7]:
def sqeuclidean_distances(x: np.ndarray, y: np.ndarray) -> float:
    return np.sum( (x - y) ** 2)
dists = jit(vmap(vmap(sqeuclidean_distances, in_axes=(None, 0)), in_axes=(0, None)))

def euclidean_distances(x: np.ndarray, y: np.ndarray) -> float:
    return np.sqrt(np.sum( (x - y) ** 2))
sqdists = jit(vmap(vmap(euclidean_distances, in_axes=(None, 0)), in_axes=(0, None)))


@jit
def matern(v1, v2, sigma = 50):
    #V1 is a [k1] vector
    #V2 is a [k2] vector
    #returns a k1xk2 matrix
    d = sqdists(v1, v2)
    #return a*np.exp(-d**2/sigma)
    return (1+np.sqrt(5)*d/sigma +5*d**2/(3*sigma**2))*np.exp(-np.sqrt(5)*d/sigma)

@jit
def exp(v1, v2, sigma):
    #V1 is a [k1] vector
    #V2 is a [k2] vector
    #returns a k1xk2 matrix
    d = dists(v1, v2)
    return np.exp(-d/sigma)
    #return (1+np.sqrt(5)*d/sigma +5*d**2/(3*sigma**2))*np.exp(-np.sqrt(5)*d/sigma)

@jit
def iq(v1, v2, sigma):
    #V1 is a [k1] vector
    #V2 is a [k2] vector
    #returns a k1xk2 matrix
    d = dists(v1, v2)
    #return a*np.exp(-d**2/sigma)
    #return (1+np.sqrt(5)*d/sigma +5*d**2/(3*sigma**2))*np.exp(-np.sqrt(5)*d/sigma)
    return 1/np.sqrt(d+sigma)

In [25]:
Ntrain = 1000
n_components = 128

In [26]:
Ytr = Outputs_fl[:Ntrain]
Xtr = Inputs_fl[:Ntrain]
pca = PCA(n_components=n_components)
Xtr = pca.fit_transform(Xtr)
Xtest = pca.transform(Inputs_fl[Ntrain:])
Ytest = Outputs_fl[Ntrain:]

In [27]:
def aux(kernel, s, nugget):
    k = kernel
    Kxx = k(Xtr, Xtr, s)
    nuggeted_matrix = Kxx.at[np.diag_indices_from(Kxx)].add(nugget)
    L = cho_factor(nuggeted_matrix)
    result = cho_solve(L, Ytr)
    Train_pred = Kxx@result #train predictions
    K_te_tr = k(Xtest, Xtr,s)
    Test_pred = K_te_tr@result #test predictions

    np.mean(np.linalg.norm(Ytr-Train_pred, axis = 1))

    aux1 = np.mean(np.linalg.norm(Ytr-Train_pred, axis = 1))
    aux2 = np.mean(np.linalg.norm(Train_pred-Ytr, axis = 1)/np.linalg.norm(Ytr, axis = 1))
    aux3 = np.mean(np.linalg.norm(Ytest-Test_pred, axis = 1))
    aux4 = np.mean(np.linalg.norm(Ytest-Test_pred, axis = 1)/np.linalg.norm(Ytest, axis = 1))
    print(s, nugget)
    print("\n Train error (abs): {0} \n Train error (rel): {1} \n Test error (abs): {2} \n Test error (rel): {3}".format(aux1, aux2, aux3, aux4))
    print('---')

In [28]:
for kernel in [matern]:
    for s in [10]:
        for nugget in [1e-8]:
            aux(kernel, s, nugget)

In [25]:
Kxx = iq(Xtr, Xtr, 10)
nuggeted_matrix = Kxx.at[np.diag_indices_from(Kxx)].add(nugget)
L = cho_factor(nuggeted_matrix)
result = cho_solve(L, Ytr)
Train_pred = Kxx@result #train predictions
K_te_tr = iq(Xtest, Xtr,10)
Test_pred = K_te_tr@result #test predictions


In [37]:
import pickle

In [39]:
pickle.dump(obj=[Ytest[42], Test_pred[42]], file = open('PredictedNS.pkl', 'wb'))

In [41]:
for kernel in [iq, matern]:
    for s in [5, 10, 20, 40, 80, 100]:
        for nugget in [1e-8]:
            aux(kernel, s, nugget)

5 1e-08

 Train error (abs): 7.276826314842877e-06 
 Train error (rel): 3.089005883597936e-06 
 Test error (abs): 0.0036650247452169396 
 Test error (rel): 0.001408151899488941
---
10 1e-08

 Train error (abs): 7.200404742330143e-05 
 Train error (rel): 3.0344639302522636e-05 
 Test error (abs): 0.003295011932740077 
 Test error (rel): 0.0012717308827894362
---
20 1e-08

 Train error (abs): 0.0005751168252746178 
 Train error (rel): 0.00023795777233394865 
 Test error (abs): 0.003558931576589692 
 Test error (rel): 0.0013742244987143153
---
40 1e-08

 Train error (abs): 0.0025715950223758775 
 Train error (rel): 0.0010307540484672058 
 Test error (abs): 0.005443076746567158 
 Test error (rel): 0.002097828482226641
---
80 1e-08

 Train error (abs): 0.007101622836243661 
 Train error (rel): 0.0027757126412045426 
 Test error (abs): 0.010035304870783596 
 Test error (rel): 0.003849685796898841
---
100 1e-08

 Train error (abs): 0.008777021698441391 
 Train error (rel): 0.00341347160174039